# 상장기업정보 수집하기

In [1]:
import pandas as pd
import time
import requests
import re
from bs4 import BeautifulSoup as bs

In [2]:
def info_extraction(trs):
    page_result = [] 
    for tr in trs: 
        stock_type = tr.select_one("td:nth-child(1) > img")['alt']
        company_name = tr.select_one("td:nth-child(1) a")['title']
        code = tr.select_one("td:nth-child(1) a")['onclick'].split("'")[1]
        code = f"{code:0<6}"
        business_type = tr.select_one("td:nth-child(2)").text
        product = tr.select_one("td:nth-child(3)").text
        regi_date = tr.select_one("td:nth-child(4)").text
        settlement = tr.select_one("td:nth-child(5)").text
        ceo = tr.select_one("td:nth-child(6)").text
        homepage = tr.select_one("td:nth-child(7) > a")['href'] if tr.select_one("td:nth-child(7) > a") != None else "홈페이지 없음"
        region = tr.select_one("td:nth-child(8)").text
        values = (stock_type, company_name, code, business_type, product, regi_date, settlement, ceo, homepage, region)
        page_result.append(values)
    return page_result

In [3]:
result = {}
cols = ['주식종목', '회사명','종목코드','업종','주요제품','상장일','결산월',
       '대표자명','홈페이지','지역']
page = 1
total_page = 1

while page <= total_page:
    print(f"{page}/{total_page} 수집중           ", end="\r")
    url ="https://kind.krx.co.kr/corpgeneral/corpList.do"
    payload = dict(method="searchCorpList", pageIndex=page, currentPageSize=100, orderMode=3, orderStat="D", searchType=13, fiscalYearEnd="all", location="all")
    r = requests.post(url, data=payload)
#     print(r.url)
#     print(r.status_code)
    soup = bs(r.content, 'lxml')
    if total_page == 1:
        total_page = int(soup.select_one(".info.type-00 > em").text.replace(",", "")) // 100 + 1

    page_result = info_extraction(soup.select("tbody > tr"))


    for items in page_result:
        for key, value in zip(cols, items):
            result.setdefault(key, []).append(value)
            
    page += 1

df = pd.DataFrame(result)



In [4]:
from io import StringIO

In [5]:
df_list = []
for page in range(1, 29):
    url ="https://kind.krx.co.kr/corpgeneral/corpList.do"
    payload = dict(method="searchCorpList", pageIndex=page, currentPageSize=100, orderMode=3, orderStat="D", searchType=13, fiscalYearEnd="all", location="all")
    r = requests.post(url, data=payload)
    temp = pd.read_html(StringIO(r.text))
    df_list.append(temp[0])
df = pd.concat(df_list)


In [6]:
df.columns = ['회사명', '업종','주요제품','상장일','결산월',
       '대표자명','홈페이지','지역']
df

,회사명,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,엔에이치스팩32호,금융 지원 서비스업,합병,2025-12-05,12월,김수한,NaN,서울특별시
1,에임드바이오,자연과학 및 공학 연구개발업,ADC 기반 항암제,2025-12-04,12월,허남구,홈페이지 보기,서울특별시
2,미래에셋비전스팩9호,금융 지원 서비스업,기업인수합병,2025-12-01,12월,황리건,NaN,서울특별시
3,삼성스팩12호,기타 금융업,기업인수합병,2025-11-28,12월,김영제,NaN,서울특별시
4,미래에셋비전스팩8호,금융 지원 서비스업,기업인수합병,2025-11-27,12월,김병철,NaN,서울특별시
...,...,...,...,...,...,...,...,...
70,유한양행,의약품 제조업,"의약품(삐콤씨, 안티푸라민, 렉라자, 로수바미브, 코푸시럽 등), 생활용품(유한락스...",1962-11-01,12월,대표이..,홈페이지 보기,서울특별시
71,CJ대한통운,도로 화물 운송업,"Contract Logistics, 포워딩, 항만하역, 해운, 택배국제특송, SCM...",1956-07-02,12월,신영수..,홈페이지 보기,서울특별시
72,경방,종합 소매업,"섬유류(면사,면혼방사,면직물,면혼방직물,화섬사,화섬직물) 제조,도매,수출입",1956-03-03,12월,"김준,..",홈페이지 보기,서울특별시
73,유수홀딩스,회사 본부 및 경영 컨설팅 서비스업,지주사업,1956-03-03,12월,송영규,홈페이지 보기,서울특별시


In [7]:
from dbio import to_db

In [8]:
# !pip install cryptography

In [9]:
to_db("stock_info_20251208","상장기업정보",df)

stock_info_20251208 데이터베이스 확인/생성 완료
